<a href="https://colab.research.google.com/github/danil31219as/AI_SQUAD/blob/main/bert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
df_train = pd.read_csv('train_stop.csv')
df_val = pd.read_csv('val_stop.csv')


In [ ]:
df_val

,Unnamed: 0,text,label
0,0,самый_ADJ первый_ADJ « остров_NOUN » архипелаг...,1
1,1,самый_ADJ первый_ADJ « остров_NOUN » архипелаг...,0
2,2,самый_ADJ первый_ADJ « остров_NOUN » архипелаг...,0
3,3,самый_ADJ первый_ADJ « остров_NOUN » архипелаг...,0
4,4,самый_ADJ первый_ADJ « остров_NOUN » архипелаг...,1
...,...,...,...
2230,2230,канадский_ADJ порноактер лука_NOUN рокко_PROPN...,0
2231,2231,канадский_ADJ порноактер лука_NOUN рокко_PROPN...,0
2232,2232,канадский_ADJ порноактер лука_NOUN рокко_PROPN...,1
2233,2233,канадский_ADJ порноактер лука_NOUN рокко_PROPN...,1


In [ ]:
X_train, y_train = df_train['text'], df_train['label']
X_val, y_val = df_val['text'], df_val['label']

In [ ]:
patterns = "[_][A-Z]*"

X_train = list(map(lambda x: re.sub(patterns, '', x), X_train))
patterns = "[!#$%&'()*+,./:;<=>?@[\]^`{|}~—\"\-»«]+"
X_train = list(map(lambda x: re.sub(patterns, '', x).replace('Q', '').replace('A', '').replace('\xad', '').replace('»', '').replace('«', ''), X_train))

In [ ]:
patterns = "[_][A-Z]*"

X_val = list(map(lambda x: re.sub(patterns, '', x), X_val))
patterns = "[!#$%&'()*+,./:;<=>?@[\]^`{|}~—\"\-»«…]+"
X_val = list(map(lambda x: re.sub(patterns, '', x).replace('Q', '').replace('A', '').replace('\xad', '').replace('»', '').replace('«', ''), X_val))

In [ ]:
def text_splitter(text, amount=512): 
    tokens = text.split(' ')
    new_text = ' '.join(tokens[-amount:])
    return new_text

In [ ]:
X_train = list(map(lambda x: text_splitter(x), X_train))
X_val = list(map(lambda x: text_splitter(x), X_val))

In [ ]:
text_splitter(X_val[0])

'самый первый  остров  архипелаг возникнуть 1923 год место соловецкий монастырь затем появиться тон  тюрьма особый назначение этап человек попадать архипелаг разный способ вагон зак баржа пароход пеший этап тюрьма арестованных доставлять  воронка   фургончик чёрного цвет роль порт архипелаг играть пересылка временной лагерь состоять палатка землянка барак участок земля открытый небо пересылка держать  политический  узда помогать специально отобранные урка  социально близкий  солженицын побывать пересылка красный пресня 1945 год эмигрант крестьянин  малый народ  перевозить красный эшелон чаща такой эшелон останавливались пустой место посреди степь тайга осуждённые сам строить лагерь особо важный заключённые основное учёные перевозиться спецконвоем перевозить солженицын назваться ядерный физик красный пресня перевезти бутырки  почему солженицын перевозить спецконвоем  перевозить особо важный заключенных'

In [ ]:
import json
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs, make_circles, make_classification, load_iris, load_digits
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import tree, metrics, model_selection
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from collections import Counter

from sklearn.model_selection import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


In [ ]:
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

In [ ]:
def clean_text(s):
    s = ''.join(re.split(r'[(]\d+[)]', s.lower()))
    
    return s

In [ ]:
def get_X_y(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append("[CLS] " + clean_text(text)+ ' [SEP] ' +query+a['text'])
                    y.append(a['label'])
    return X, y

In [ ]:
X_train, y_train = get_X_y('train.jsonl')

In [ ]:
X_val, y_val = get_X_y('val.jsonl')

In [ ]:
X_test, y = get_X_y('test.jsonl')

In [ ]:
X_train

In [ ]:
def update_text_sep(x_train):
  s = []
  for elem in x_train:
    s.append('[CLS] ' + elem.replace('QUERY:', '[SEP]').replace('ANS: ', ''))
  return s

In [ ]:
X_train = update_text_sep(X_train)

In [ ]:
len(X_train[0].split())

166

In [ ]:
train_df = pd.DataFrame({
    'text': X_train,
    'label':y_train
})

print(train_df.head())

eval_df = pd.DataFrame({
    'text': X_val,
    'label': y_val
})

print(eval_df.head())

                                                text  label
0  человек не мочь существовать природа поэтому п...      1
1  человек не мочь существовать природа поэтому п...      0
2  человек не мочь существовать природа поэтому п...      0
3  человек не мочь существовать природа поэтому п...      1
4  человек не мочь существовать природа поэтому п...      0
                                                text  label
0  самый первый  остров  архипелаг возникнуть 192...      1
1  самый первый  остров  архипелаг возникнуть 192...      0
2  самый первый  остров  архипелаг возникнуть 192...      0
3  самый первый  остров  архипелаг возникнуть 192...      0
4  самый первый  остров  архипелаг возникнуть 192...      1


In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
'DeepPavlov/bert-base-multilingual-cased-sentence'
'DeepPavlov/rubert-base-uncased'

In [ ]:
import tqdm

In [ ]:
!pip create -n st python pandas tqdm
!pip activate st

ERROR: unknown command "create"
ERROR: unknown command "activate"


In [ ]:
train_args = {'num_train_epochs': 2, 'max_seq_length': 512,'learning_rate': 1e-5, 'overwrite_output_dir': True, 'gradient_accumulation_steps': 5, 'use_multiprocessing': False}

In [ ]:
model = ClassificationModel('bert', 'bert-base-multilingual-uncased', use_cuda=True, args=train_args)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
# Create a TransformerModel


# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:391: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/11950 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1494 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 2:   0%|          | 0/1494 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:970: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2235 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/280 [00:00<?, ?it/s]

In [ ]:
best_f1 = 0
best_pred = 0
best_rate = 0
for rate in [4e-4, 5e-5, 6e-6]:
  train_args = {'num_train_epochs': 2, 'train_batch_size': 16, 'eval_batch_size': 32, 'max_seq_length': 512,'learning_rate': rate, 'save_steps': 50000, 'overwrite_output_dir': True, 'gradient_accumulation_steps': 5, 'use_multiprocessing': False, 'manual_seed': 42}
  model = ClassificationModel('bert', 'bert-base-multilingual-uncased', use_cuda=True, args=train_args)
  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(eval_df)
  predictions, raw_outputs = model.predict(X_val)
  if f1_score(y_val, predictions) > best_f1:
    best_f1 = f1_score(y_val, predictions)
    best_rate = rate
    best_pred, raw = model.predict(X_test)
print(best_f1, best_rate)
save_output(best_pred, 'bert_test_best_rate.jsonl')
from google.colab import files
files.download('bert_test_best_rate.jsonl') 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

  0%|          | 0/11950 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/747 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

  0%|          | 0/2235 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

0.6822244289970207 5e-05


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle

# save
with open('model_bert.pkl','wb') as f:
    pickle.dump(model,f)


In [ ]:

# load
with open('model.pkl', 'rb') as f:
    clf2 = pickle.load(f)

In [ ]:
predictions, raw_outputs = model.predict(X_val)

  0%|          | 0/2235 [00:00<?, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

In [ ]:
raw_outputs[:20]

array([[ 0.50878906, -0.63037109],
       [-0.17114258,  0.04031372],
       [ 0.25024414, -0.46142578],
       [-0.80078125,  0.66162109],
       [-0.53320312,  0.39868164],
       [ 0.79003906, -0.69140625],
       [-0.4987793 ,  0.47216797],
       [ 0.66113281, -0.69775391],
       [ 0.82617188, -0.72363281],
       [ 0.81591797, -0.70947266],
       [ 0.76220703, -0.69384766],
       [ 0.13977051, -0.31225586],
       [-0.13171387,  0.00914764],
       [-0.02882385, -0.10174561],
       [-0.18933105,  0.06216431],
       [-0.14074707,  0.03436279],
       [-0.16784668,  0.05419922],
       [-0.03649902, -0.07568359],
       [-0.06762695, -0.04611206],
       [ 0.24609375, -0.40869141]])

In [ ]:
f1_score(y_val, predictions)

0.6715542521994136

In [ ]:
def qa_predict(predictions, json_test):
  state = []
  i = 0
  d_count = {2: 1, 3: 1, 4:2, 5: 2}
  for pas in list(map(lambda tr: json.loads(tr), json_test)):
    for question in pas['passage']['questions']:
      answers = []
      for answer in question['answers']:

        answers.append(predictions[i][0])
        i += 1
      if len(answers) in d_count:
        count = d_count[len(answers)]
      else:
        count = 3
      
      border = sorted(answers)[count - 1]

      for i in range(len(answers)):
        if answers[i] <= border:
          answers[i] = 1
        else:
          answers[i] = 0
      state.extend(answers)
  return state

In [ ]:
with open('val.jsonl', 'r') as json_file:
  val_list = list(json_file)

In [ ]:
with open('test.jsonl', 'r') as json_file:
  test_list = list(json_file)

In [ ]:
f1_score(y_val, qa_predict(raw_outputs, val_list))

0.4790528233151184

In [ ]:
df_test = pd.read_csv('test_stop.csv')

In [ ]:
X_test = df_test['text']

In [ ]:
patterns = "[_][A-Z]*"

X_test = list(map(lambda x: re.sub(patterns, '', x), X_test))
patterns = "[!#$%&'()*+,./:;<=>?@[\]^`{|}~—\"\-»«]+"
X_test = list(map(lambda x: re.sub(patterns, '', x).replace('Q', '[SEP]').replace('A', '').replace('\xad', '').replace('»', '').replace('«', ''), X_test))

In [ ]:
def text_splitter(text, amount=512): 
    tokens = text.split(' ')
    new_text = ' '.join(tokens[-amount:])
    return new_text

In [ ]:
X_test = list(map(lambda x: text_splitter(x), X_test))


In [ ]:
predictions, raw_outputs = model.predict(X_test)

  0%|          | 0/7614 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

In [ ]:
raw_outputs[:10]

array([[ 0.71191406, -0.75634766],
       [-0.67089844,  0.60644531],
       [-1.01855469,  0.93115234],
       [-1.32421875,  1.23339844],
       [ 0.69335938, -0.77050781],
       [ 0.69335938, -0.77050781],
       [ 0.28173828, -0.38867188],
       [-0.03305054, -0.05197144],
       [ 1.20898438, -1.27148438],
       [-1.61425781,  1.64160156]])

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
def save_output(y_pred, path):
    
    with open('bert_test_4.jsonl') as file:
      data = list(jsonlines.Reader(file))
    index = 0
    for i, elem in enumerate(data):
      for j, query in enumerate(elem['passage']['questions']):
        for k, ans in enumerate(query['answers']):
          data[i]['passage']['questions'][j]['answers'][k]['label'] = int(y_pred[index])
          index += 1
    with open(path, mode="w") as file:
        for line in sorted(data, key=lambda x: int(x.get("idx"))):
            line["idx"] = int(line["idx"])
            file.write(f"{json.dumps(line, ensure_ascii=False)}\n")

In [ ]:
save_output(qa_predict(raw_outputs, test_list), 'bert_test_3_2.jsonl')

In [ ]:
save_output(predictions, 'bert_stop.jsonl')

In [ ]:
save_output(data, path='cb_qa.jsonl')

In [ ]:
import jsonlines
with open('bert_stop.jsonl') as file:
    data = list(jsonlines.Reader(file))
    index = 0
    for i, elem in enumerate(data):
      for j, query in enumerate(elem['passage']['questions']):
        for k, ans in enumerate(query['answers']):
          if index in indexes:
            data[i]['passage']['questions'][j]['answers'][k]['label'] = 0

          index += 1

In [ ]:
indexes = [1009,
 1011,
 1012,
 1014,
 1015,
 1517,
 1518,
 1520,
 1524,
 1525,
 1526,
 1529,
 1530,
 1627,
 1628,
 1630,
 1631,
 1632,
 1635,
 1637,
 1639,
 1640,
 1642,
 1643,
 1645,
 1646,
 1648,
 1649,
 1652,
 1653,
 1654,
 1657,
 1658,
 1660,
 1661,
 1664,
 1665,
 1666,
 1667,
 1669,
 1672,
 1674,
 1675]

In [ ]:
import json
with open('bert_stop.jsonl', mode="w") as file:
        for line in sorted(data, key=lambda x: int(x.get("idx"))):
            line["idx"] = int(line["idx"])
            file.write(f"{json.dumps(line, ensure_ascii=False)}\n")